# Data cleaning

**Dataset used**
- open_iof_20min_dataset.csv

In [ ]:
# Data links
data_url = {
    'iof_data_1min_csv' : "https://drive.google.com/uc?id=1_jYVXj7mt8Zzpjn8WGI111G-kWRTbfjU",
    'iof_data_1min_parq' : "https://drive.google.com/uc?id=1j5SS136UzbSPu8TqG9RRUMi6-wWF9dzq",
    'mixingTank' :  "https://drive.google.com/uc?id=1b5Qn5LIa6KAE03Tq4yRVdhTyUmZLxRjt",
    'moons' : "https://drive.google.com/uc?id=1a9zTkPEpuHGj6LzGzuLe-JSLg_4GJef4",
    'open_iof_20min' : "https://drive.google.com/uc?id=15lkhdBfWnjlpgpEx4T2XcRApKr-dmBb0",
    'open_iof_cleaned' : "https://drive.google.com/uc?id=1WVbJvYsGy-iKlsW4WaDZrKy_NhK2tJLW",
}

In [ ]:
# Import packages
import pandas as pd                 
import matplotlib.pyplot as plt     
import numpy as np                  

In [ ]:
# pandas display format: two decimals
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
# Ingest data
fname = data_url['open_iof_20min']
indexColumn = 'date' 
dateColumns = ['date']
df = pd.read_csv(fname, index_col=indexColumn, parse_dates=dateColumns) 

In [ ]:
# Look at current column names
df.columns

In [ ]:
# Rename columns
## Rename columns (variables) for easier interpretation
# Create a dictionory which contains the old names and new names
# Format: dictionary = {'old_name_variable_1': 'new_name_variable_1', 'old_name_variable_2': 'new_name_variable_2'}
old_comp_names = ['plant.feed.iron.comp', 
                'plant.feed.silica.comp',
                'plant.filters.product.iron.comp', 
                'plant.filters.product.silica.comp']
new_comp_names = ['d feed iron %',
                'd feed silica %',
                'y product iron %',
                'y product silica %']

old_sump_names = ['plant.flotation.sump01.starch.flow',
                'plant.flotation.sump01.amina.flow',
                'plant.flotation.sump01.discharge.flow',
                'plant.flotation.sump01.discharge.ph',
                'plant.flotation.sump01.discharge.density']

new_sump_names = ['u starch m3/h',
                'u amina m3/h',
                'd feed m3/h',
                'd feed ph',
                'd feed SG']

old_air_flow_names = [name for name in df.columns if 'air.flow' in name]
new_air_flow_names = [name.replace('plant.flotation.bank01.column','x C') + ' Nm3/h' for name in old_air_flow_names]

old_depth_names = [name for name in df.columns if 'froth' in name]
new_depth_names = [name.replace('plant.flotation.bank01.column','x C') + ' mm' for name in old_depth_names]

old_names = old_comp_names+old_sump_names+old_air_flow_names+old_depth_names
new_names = new_comp_names+new_sump_names+new_air_flow_names+new_depth_names

nameChangeDictionary = dict(zip(old_names,new_names))
nameChangeDictionary

In [ ]:
# Rename columns in the data frame according to the specified dictionary
df.rename(columns=nameChangeDictionary, inplace=True)
# Look at the new column names
df.columns

In [ ]:
# Basic statistics
df.describe().T

## Outliers: .loc()
The slicing/indexing function with assignment can be used to deal with outliers:

Identify outlier rows and replace outliers with missing values (`np.nan`). Missing values can then be handled as discussed further below. 

In [ ]:
# Outlier detection: Knowledge-based (visual highlighting)
df.plot(subplots=True,figsize=(10,50),marker='.')
# Domain-knowledge prompts: 
# air flow < 200 Nm3/h
# froth depth < 200 mm

In [ ]:
# Outlier detection:
# Assigning missing values to rows specified as containing outliers
outlierRowsAirFlow01 = df['x C01.air.flow Nm3/h'] < 200
df.loc[outlierRowsAirFlow01,'x C01.air.flow Nm3/h'] = np.nan
df.describe().T

In [ ]:
# Outlier detection
df['x C01.air.flow Nm3/h'].plot(marker='.')

In [ ]:
# Remove outliers from more columns
for name in new_air_flow_names:
    df.loc[df[name]<200,name] = np.nan

In [ ]:
# Check statistics
df.describe().T

# 🎯 Practice Point
Remove outliers from 'x C04.froth.depth mm' where the values are above 600. Display the basic statistics of the data frame or plot the froth depth data to check if your outlier removal worked.

## Missing values: .fillna()
Missing values (represented by `np.nan`) can be replaced by fixed values, e.g., the mean or median of the variable in question. The method `.fillna()` can be used to replace `np.nan` with a fixed value.

`.ffill()` and `.bfill()` can also be used for forward filling or backward filling, instead of a fixed value.

In [ ]:
# Missing values: Filling with fixed value
df['d feed iron % (mean replacement)'] = df['d feed iron %'].fillna(df['d feed iron %'].mean())
df['d feed iron % (mean replacement)'].plot(marker='.',figsize=(12,4))

In [ ]:
# Missing values: Forward fill
df['d feed iron % (forward fill)'] = df['d feed iron %'].ffill()
df['d feed iron % (forward fill)'].plot(marker='.',figsize=(12,4))

In [ ]:
# Missing values: Backward fill
df['d feed iron % (backward fill)'] = df['d feed iron %'].bfill()
df['d feed iron % (backward fill)'].plot(marker='.',figsize=(12,4))

In [ ]:
# Missing values: Interpolation (linear, time reference)
df['d feed iron % (interpolated)'] = df['d feed iron %'].interpolate(method='time')
df['d feed iron % (interpolated)'].plot(marker='.',figsize=(12,4))

# 🎯 Practice Point
Use different methods to fill 'd feed silica %' missing values. Show the result of the data cleaning by plotting the filled variable in a time series.

In [ ]:
# Missing values: Final replacement decision
df['d feed iron %'] = df['d feed iron %'].bfill()
df['d feed silica %'] = df['d feed silica %'].bfill()

In [ ]:
# Drop demonstraton variables from data frame
variables_to_delete = ['d feed iron % (mean replacement)',
    'd feed iron % (forward fill)',
    'd feed iron % (backward fill)',
    'd feed iron % (interpolated)']
df.drop(labels=variables_to_delete,axis=1,inplace=True)
df.info()

## Missing values: Understanding the size of the problem

In [ ]:
# Count missing value occurences in a variable
print("Missing data percent:")
for vname in df.columns:
    missingpercent = 100*df[vname].isnull().sum()/df.shape[0]
    if missingpercent > 0:
        print(f"{missingpercent:4.1f} % missing in {vname}")
        # Unpacking {missingpercent:4.1f} 
        # -- missingpercent = number to display
        # -- : = indicates that formatting of number will follow
        # -- 4 = width in characters allocated for display of number (can be longer)
        # -- . = indicates decimal point
        # -- 1 = indicates number of digits after decimal point
        # -- f = indicates variable type, a floating point number in this case

## Noise removal: .rolling()
Rolling window calculations can be done on data frames. For a specific row, a window (i.e., fixed number of preceding rows) is considered, and a calculation done on values on that window. The result is stored in that row, and then the window calculation is repeated for the next row.

The sample time is specified in special string notation, with format `[number][unit]`, e.g., `'10S'`. Units include:
- 'd': day frequency
- 'h': hour frequency
- 'min': minute frequency
- 's': second frequency

For more units, see [this link](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases).

When `.rolling()` is applied, the sampling frequency of the data frame stays the same.

In [ ]:
# Noise removal: Filtering
# Visualize noisy variable
df['u starch m3/h'].plot(marker='.',ls='none',figsize=(12,4))

In [ ]:
# Noise removal: Filtering
# Rolling average
df['u starch m3/h (filtered)'] = df['u starch m3/h'].rolling('1d').mean()
# Visualize smoothed variable
df['u starch m3/h (filtered)'].plot(marker='.',ls='none',figsize=(12,4))

# 🎯 Practice Point
Play around with the size of the rolling window (e.g., 1 hour, 1 day, 1 week) and the type of summary statistic (e.g., mean, median, maximum, sum) to determine a good tradeoff of rolling window size as well as summary statistic. Visualize your result in each case.

## Down/upsampling: Resampling
For time series data, the `.resample()` method can be used to change the sample time of the data. For example, if the data was original captured at 20 second intervals, but a rolled up sample time of 1 hour is of interest, `.resample()` can be used to aggregate the data to 1 hour intervals.

**Downsampling** involves going from a high sampling frequency (short time between samples) to a low sampling frequency (long time between samples). This implies that some aggregate statistic needs to be calculated to transform the many samples of the high frequency data to the fewer samples of the low frequency result. Operations like `.mean()` or `.median()` can be used.

When `.resample()` is applied, the sampling frequency of the data frame changes. 

**Upsampling** involves going from a low sampling frequency to a high sampling frequency, typically when joining more than one dataframe. This will result in missing values (`NaN`), since it is unknown what happened between observed values. Missing value replacement can be used to estimate ("guess") what happed at a higher frequency.

In [ ]:
# Downsampling: Median approach
# (downsampled to hourly values)
dfIOFHourly = df.resample('1h').median()
dfIOFHourly.head()

In [ ]:
# Downsampling: Median approach
dfIOFHourly.plot(subplots=True,marker='.',figsize=(10,50))

In [ ]:
# Upsampling: Combining two data frames and upsampling low frequency data
# Create example data frames with certain time range and sampling frequency
# (note: this would typically be done by importing from csv/xlsx)
# Low frequency measurement
date_range1 = pd.date_range(start='2025-05-01', periods=30, freq='h')
df1 = pd.DataFrame({
    'timestamp': date_range1,
    'Product quality': 0.2*np.random.randn(30),
})
df1.set_index('timestamp', inplace=True)
df1.plot(marker='.')
# High frequency measurement
date_range2 = pd.date_range(start='2025-05-01', periods=30*60, freq='min')
df2 = pd.DataFrame({
    'timestamp': date_range2,
    'Temperature': 70+5*np.random.randn(30*60),
})
df2.set_index('timestamp', inplace=True)
df2.plot(marker='.')

In [ ]:
# Combine two dataframes
df_combined = pd.concat([df1,df2],axis=1)
df_combined.plot(subplots=True,marker='.')

In [ ]:
# Upsampling (backfill)
df_combined.bfill().plot(subplots=True,marker='.')

In [ ]:
# Downsampling
df_combined.resample('1h').median().plot(subplots=True,marker='.')

## Missing values: .dropna()
List-wise deletion of missing values (i.e., removing rows with any missing values in it) can be done with the `.dropna()` method. To ensure that the data frame is modified, the argument `inplace=True` is included.

In [ ]:
## Missing values: List-wise deletion
df.dropna(inplace=True)
df.info()

In [ ]:
# Export clean dataset
df.to_csv('../data/open_data_cleaned.csv')